In [6]:
%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
from pathlib import Path
from research_tools.gpu import get_gpus_available

load_dotenv()


hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

n_gpus = 1

gpus_available = get_gpus_available()
n_gpus = min(n_gpus, len(gpus_available))
gpus = gpus_available[:n_gpus]

assert n_gpus > 0, "No GPUs available"

os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in gpus])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Optional
import torch
from research_tools.utils import set_seed

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "LLM-LAT/zephyr7b-beta-rmu-lat-unlearn-wmdp-bio-cyber"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3-8B"
model_id = "HuggingFaceH4/zephyr-7b-beta"

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

/mnt/align4_drive/tcqian/unlearn_order/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:00<00:00,  9.44it/s]
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/chat_template.jinja'


In [5]:
path = Path("./models/test")
os.makedirs(path, exist_ok=True)
model.save_pretrained(path)